## Counting Rubik's Snake shapes, up to reversal symmetry

*Dmytro Fedoriaka, August 2024*

This is an addition to the [main notebook](count-shapes.ipynb). 

Here I am computing one of sequences defined in that notebook. Namely, number of Rubik's Snake shapes *up to reversal*. That is, we count snakes without distinguishing tail and head. Let's denote this sequence $D_n$.

It tuns out that we can use pre-computed sequence $S_n$ (see the main notebook) to compute $D_n$ with much faster asymptotic, $O(2^n \cdot n)$ instead of $O(4^n)$.

Denote $F_n$ - the set of all shapes that are mapped to themselves by reversal. Then $2 D_n = S_n+F_n$. From this, $D_n = (S_n +F_n)/2$
 * This can be proven by counting argument. Consider all strings from a set corresponding to $D_n$, and add to this set all reverses. In resulting multiset we will get all strings corresponding to $S_n$, but some of the strings will appear twice. These are exactly those that are fixed point for reversal.
 * Alternatively, this follows from Burnside's lemma.

How to compute $F_n$? These are exactly those valid shapes whose formula is a palindrome. So we can explicitly enumerate all palindromes of length $n-1$ with characters 0,1,2,3 - there are $4^{[n/2]} = O(2^n)$ of them. Then we can directly check whether each of them is a formula describing a valid shapes. The complexity is $O(2^n \cdot n)$.


In [1]:
from rubiks_snake import RubiksSnakeCounter
import time 

t0=time.time()
D = [0]*(27)
for n in range(1,27):
  S = RubiksSnakeCounter.S[n]
  F = RubiksSnakeCounter.count_palindrome_shapes(n)
  D[n] = (F+S)//2
  print(f"S[{n}]={S}, F[{n}]={F}, D[{n}]={D[n]}")
print("Total time: %fs." % (time.time()-t0))
print("Answer: D=", D[1:])

S[1]=1, F[1]=1, D[1]=1
S[2]=4, F[2]=4, D[2]=4
S[3]=16, F[3]=4, D[3]=10
S[4]=64, F[4]=16, D[4]=40
S[5]=241, F[5]=13, D[5]=127
S[6]=920, F[6]=60, D[6]=490
S[7]=3384, F[7]=52, D[7]=1718
S[8]=12585, F[8]=221, D[8]=6403
S[9]=46471, F[9]=185, D[9]=23328
S[10]=172226, F[10]=802, D[10]=86514
S[11]=633138, F[11]=700, D[11]=316919
S[12]=2333757, F[12]=2957, D[12]=1168357
S[13]=8561679, F[13]=2483, D[13]=4282081
S[14]=31462176, F[14]=10820, D[14]=15736498
S[15]=115247629, F[15]=9199, D[15]=57628414
S[16]=422677188, F[16]=39608, D[16]=211358398
S[17]=1546186675, F[17]=33105, D[17]=773109890
S[18]=5661378449, F[18]=144593, D[18]=2830761521
S[19]=20689242550, F[19]=122038, D[19]=10344682294
S[20]=75663420126, F[20]=527782, D[20]=37831973954
S[21]=276279455583, F[21]=439415, D[21]=138139947499
S[22]=1009416896015, F[22]=1922239, D[22]=504709409127
S[23]=3683274847187, F[23]=1613723, D[23]=1841638230455
S[24]=13446591920995, F[24]=7005651, D[24]=6723299463323
S[25]=49037278586475, F[25]=5817729, D[25]

Validation:
* First 14 terms match with first terms computed with [snek](https://github.com/scholtes/snek).
* I also wrote my own program explicitly enumerating all valid states and doing deduplication, see [here](scracth.ipynb). I used it to compute the first 12 terms and results match.
* However, my value for $D_{24}$ is 6723299463323, which differs from value found [here](https://blog.ylett.com/2011/09/rubiks-snake-combinations.html) by 0.02%.